In [0]:
from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import time

import urllib
import json
from pprint import pprint
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import parse, Element

import os
os.chdir("/gdrive/My Drive/Colab Notebooks/dacon")


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
train = pd.read_csv("./data/comp_9th/funda_train.csv", parse_dates=['transacted_date'])

train["Time"] = pd.to_timedelta(train["transacted_time"]+":00")
train["t_DateTime"] = train["transacted_date"] + train["Time"]
train['t_time_interval'] = (train['t_DateTime'] - train['t_DateTime'].shift(1))

train_cp = train.copy()

# 1. Reducing negative value



In [0]:
col2num = {'index': 0}
cols = train_cp.columns.to_list()
for i in range(1,len(cols)+1):
    col2num[cols[i-1]] = i

In [0]:

# 'index': 0, 'store_id': 1, 'card_id': 2, 'card_company': 3, 'transacted_date': 4, 
# 'transacted_time': 5, 'installment_term': 6, 'region': 7, 'type_of_business': 8, 
# 'amount': 9, 'Time': 10, 't_DateTime': 11, 't_time_interval': 12



def reduce_noise_by_removing_neg_vals(df_copy):
    df_pos = df_copy[df_copy.amount > 0]
    df_neg = df_copy[df_copy.amount < 0]
    no_match_cnt = 0
    matched_exact = 0
    modified_amt =0
    cnt = 0
    start = datetime.now()
    print(start)
    print("length_of_neg: "+str(len(df_neg.to_records()[:])))
    start_time = time.time()
    list_a = []
    list_b = []
    list_c = []
    
    for nega_i in df_neg.to_records()[:]:    
        cnt+=1
        
        store_i = nega_i[col2num['store_id']]
        date_i = nega_i[col2num['t_DateTime']]
        card_i = nega_i[col2num['card_id']]
        card_c = nega_i[col2num['card_company']]
        amt_i = nega_i[col2num['amount']]
        idx_i = nega_i[col2num['index']]
        
        row_i = df_pos[df_pos.store_id == store_i]
        row_i = row_i[row_i.amount >= abs(amt_i)]
        row_i = row_i[row_i.t_DateTime <= date_i]
        row_i = row_i[row_i.index < idx_i] # 추가
       
        row_i_hard = row_i[row_i.card_id == card_i]
        row_i_hard = row_i_hard[row_i_hard.card_company == card_c]
        row_i_hard = row_i_hard[row_i_hard.amount == abs(amt_i)]
        row_i_hard = row_i_hard[~row_i_hard.index.isin(list_a)]
        
        if len(row_i_hard) > 0:
            row_i_hard = row_i_hard[row_i_hard.t_DateTime == max(row_i_hard.t_DateTime)]
            row_i_hard = row_i_hard[row_i_hard.index == max(row_i_hard.index)]

            list_a.append(row_i_hard.index.item())
            matched_exact += 1

        elif len(row_i) > 0:
            row_i = row_i[row_i.t_DateTime == max(row_i.t_DateTime)]
            row_i = row_i[row_i.index == max(row_i.index)]
            
            if (date_i - row_i.t_DateTime.values[0]) > np.timedelta64(7, 'D'):
                list_c.append(nega_i[col2num['index']])
                no_match_cnt += 1

            else:            
                list_b.append((row_i.index.item(), idx_i, row_i.amount.item()+amt_i, amt_i, row_i.amount.item() , row_i.t_DateTime.values[0], date_i )) # 바뀌어야할 idx,바뀌어야할 amt

                df_pos.set_value(row_i.index,'amount', row_i.amount + amt_i)
                modified_amt += 1
        else:
            list_c.append(nega_i[col2num['index']])
            no_match_cnt += 1
                    
        if cnt % 1000 == 0:
            print(cnt)
            print("--- %s seconds ---" % (time.time() - start_time))
            start_time = time.time()
        
    end = datetime.now()
    time_took = (end - start).seconds / 60
    print(round(time_took, 2))
    print("row matched exact: " + str(matched_exact) )
    print("row matched amt: " + str(modified_amt) )
    print("row no matches: " + str(no_match_cnt) )
    return df_pos, matched_exact, modified_amt, no_match_cnt, list_a, list_b, list_c


In [0]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

reduced_train, a,b,c, t1,t2,t3 = reduce_noise_by_removing_neg_vals(train_cp)

2019-07-27 08:58:58.312932
length_of_neg: 73100
1000
--- 29.69374680519104 seconds ---
2000
--- 28.552396535873413 seconds ---
3000
--- 28.66165542602539 seconds ---
4000
--- 29.16180443763733 seconds ---
5000
--- 28.470367431640625 seconds ---
6000
--- 30.515324354171753 seconds ---
7000
--- 29.523693799972534 seconds ---
8000
--- 28.852576971054077 seconds ---
9000
--- 28.983911514282227 seconds ---
10000
--- 29.884563446044922 seconds ---
11000
--- 30.35585880279541 seconds ---
12000
--- 29.891977548599243 seconds ---
13000
--- 32.1201388835907 seconds ---
14000
--- 28.960426330566406 seconds ---
15000
--- 30.47464084625244 seconds ---
16000
--- 30.623291492462158 seconds ---
17000
--- 30.355305433273315 seconds ---
18000
--- 32.0415358543396 seconds ---
19000
--- 30.05998468399048 seconds ---
20000
--- 29.462265729904175 seconds ---
21000
--- 31.153777599334717 seconds ---
22000
--- 31.377310514450073 seconds ---
23000
--- 31.013768911361694 seconds ---
24000
--- 29.630330562591553

In [0]:
# was: exact match was 63922
pd.options.mode.chained_assignment = 'warn' # default='warn'

In [0]:
print(reduced_train.shape[0] - len(t1))
reduced_train[~reduced_train.index.isin(t1)].shape[0]

6419526


6419526

In [0]:
train_cp['store_id'][train_cp['amount'] == 0 ].count()

80

In [0]:
print(t2[0:1])
reduced_train[reduced_train.index == 8610]

[(8610, 8611, 15000.00000000001, -7857.142857142857, 22857.142857142866, numpy.datetime64('2018-05-27T17:40:00.000000000'), numpy.datetime64('2018-05-27T19:09:00.000000000'))]


,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,Time,t_DateTime,t_time_interval
8610,2,3780,b,2018-05-27,17:40,0,NaN,NaN,15000.0,17:40:00,2018-05-27 17:40:00,00:05:00


In [0]:
fully_dropped = reduced_train.drop(t1)

In [0]:
fully_dropped.to_csv('./data/comp_9th/fully_dropped.csv')


# 2. Korean Holiday Data


In [0]:
def query_sender(key, year, month): 
    url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear="
    m = "&solMonth="+month
    k = "&ServiceKey="+key
    
    request = urllib.request.Request(url+year+m+k) 
    request.get_method = lambda: 'GET' 
    response_body = urllib.request.urlopen(request).read() 
    u = str(response_body, "utf-8") 
    return u

def xml_to_item_list(xml_string):
    result = []
    root = ET.fromstring(xml_string)
    elements = root.findall('body/items/item')
    for item in elements:
        try:
            item_list = []
            item_list.append(item.find('locdate').text)
            item_list.append(item.find('dateName').text)
            result.append(item_list)
        except Exception as e:
            print("This row will be ignored. ",item_list)            
    return result 

In [0]:

ServiceKey = "MYSERVICEKEY"
years = [str(i) for i in  range(2016,2020)]
months = [str(i) if len(str(i)) == 2   else "0"+str(i) for i in range(1,13)]
holiday = []
for y in years:
    for m in months:
        xml_str =query_sender(ServiceKey,y,m)
        holi_info = xml_to_item_list(xml_str)
        holiday.extend(holi_info)

In [0]:
df = pd.DataFrame(holiday)
df.columns= ['h_date','day_info']
df.to_csv("./data/comp_9th/holiday.csv",index=False)

In [0]:
pd.read_csv("./data/comp_9th/holiday.csv")

,h_date,day_info
0,20160101,신정
1,20160207,설날
2,20160208,설날
3,20160209,설날
4,20160210,대체공휴일
5,20160301,삼일절
6,20160413,국회의원선거일
7,20160505,어린이날
8,20160514,석가탄신일
9,20160606,현충일


# 3. Store_info

In [0]:
def mask_first(x):
    result = np.ones_like(x)
    result[0] = 0
    return result
  
def max_23(x):
    result = np.ones_like(x)
    #result[0] = 1
    result[1] = 0
    result[2] = 0
    return result

def make_store_interval_info(df):

    # interval 없는 첫행 지우기
    masked = df.groupby(['store_id'])['store_id'].transform(mask_first).astype(bool)
    train_cp = df.loc[masked]
    max_interval_check = train_cp[['store_id','t_time_interval','type_of_business']].groupby('store_id').max()
    max_interval_check.columns = ['max_interval', 'type_of_business']

    train_cp = train_cp.sort_values(['store_id','t_time_interval'], ascending=False)

    check = train_cp.groupby(['store_id'])['store_id'].transform(max_23).astype(bool)
    interval_23 = train_cp.loc[~check]
    interval_23 = pd.pivot_table(interval_23, index='store_id', values='t_time_interval', aggfunc=['max','min'])
    max_2 = interval_23['max']
    max_2.columns = ['2nd_max']
    max_3 = interval_23['min']
    max_3.columns = ['3rd_max']
    #print(max_interval_check.shape)
    interval_check = pd.merge(max_interval_check, max_2, on = 'store_id', how ='left')
    interval_check = pd.merge(interval_check, max_3, on = 'store_id', how ='left')
    #print(interval_check.shape)
    interval_check = interval_check[['type_of_business','max_interval', '2nd_max', '3rd_max']]
    interval_check.columns = ['type_of_business','1st_max', '2nd_max', '3rd_max']
    interval_check = interval_check.sort_values('1st_max',ascending=False)
    #interval_check.head(50)

    # masked = train_raw.groupby(['store_id'])['store_id'].transform(mask_first).astype(bool)
    # train_cp = train_raw.loc[masked]

    train_cp = train_cp[['store_id','t_time_interval']]
    train_cp['t_time_interval']= train_cp['t_time_interval']/ np.timedelta64(1,'m')
    interval_stats_M = train_cp.groupby('store_id')\
                          .agg(['mean','median','std',lambda x : np.percentile(x,75) ])\
                          .reset_index()
    interval_stats_M.columns = ['store_id','interval_M_mean','interval_M_median','interval_M_std','interval_M_75']


    store_date = train_raw[['store_id','transacted_date','amount']].groupby(['store_id','transacted_date']).sum().reset_index()
    store_date['nxt_date'] = store_date.groupby(['store_id'])['transacted_date'].shift(-1)

    store_date['date_interval'] = store_date['nxt_date'] - store_date['transacted_date']

    store_date = store_date[['store_id','date_interval']]
    store_date['date_interval']= store_date['date_interval']/ np.timedelta64(1,'D')
    interval_stats_D = store_date.groupby('store_id')\
                        .agg(['mean','median','std'])\
                        .reset_index()
    interval_stats_D.columns = ['store_id','interval_D_mean','interval_D_median','interval_D_std']

    interval_check = pd.merge(interval_check, interval_stats_M, on = 'store_id', how ='left')
    interval_check = pd.merge(interval_check, interval_stats_D, on = 'store_id', how ='left')

    return interval_check



train_raw = pd.read_csv("./data/comp_9th/fully_dropped.csv", parse_dates=['transacted_date','t_DateTime'], index_col=0)
train_raw['Time'] = pd.to_timedelta(train_raw['Time'])
train_raw['t_time_interval'] = pd.to_timedelta(train_raw['t_time_interval'])

store_info = make_store_interval_info(train_raw)

new_train = train_raw.copy()

# new_train.head()

store_date = new_train[['store_id','transacted_date','amount']].groupby(['store_id','transacted_date']).sum().reset_index()
store_date['nxt_date'] = store_date.groupby(['store_id'])['transacted_date'].shift(-1)
store_date['date_interval'] = store_date['nxt_date'] - store_date['transacted_date']
store_date['date_interval'] = store_date['date_interval'].fillna(value = 0)

store_date['date_interval'] = store_date['date_interval'] / np.timedelta64(1, 'D')
store_date.head()

MAX_INTERVAL_DATE = [6,13,27]

def make_cycle(df):
    for i in MAX_INTERVAL_DATE:

        df['is_closed'] = df['date_interval'].map(lambda x: True if ((x == 0.) or (x > i)) else False  )

        df[f'cycle_idx_week{int((i+1)/7)}'] = (df.groupby('store_id')['is_closed']
                                                  .cumsum()
                                                  .shift(1)
                                                  .fillna(0)
                                                  .astype(int)
                                                    )

        head_index = df.groupby('store_id')[f'cycle_idx_week{int((i+1)/7)}'].head(1).index
        df.loc[head_index, f'cycle_idx_week{int((i+1)/7)}'] = 0 

    return df

store_date = make_cycle(store_date)
trDate_store = new_train.groupby(['store_id'])['transacted_date'].agg(['max','min']).reset_index()
trDate_store.head()

store_date = store_date.pivot_table(index='store_id', values=['cycle_idx_week1', 'cycle_idx_week2', 'cycle_idx_week4' ], aggfunc='max').reset_index()
# temp['cycle_idx'].value_counts()

store_date=store_date.merge(trDate_store,on='store_id', how= 'left')

store_info = store_date.merge(store_info,on = 'store_id', how= 'left')

store_info['1st_max'] = store_info['1st_max'] / np.timedelta64(1, 'h')
store_info['2nd_max'] = store_info['2nd_max'] / np.timedelta64(1, 'h')
store_info['3rd_max'] = store_info['3rd_max'] / np.timedelta64(1, 'h')



In [0]:
store_info['tr_len'] = (store_info['max']-store_info['min']) / np.timedelta64(1, 'D')

In [0]:
# store_info.drop(['max',	'min',	'type_of_business'], axis=1, inplace=True)

In [0]:
store_info.head()

,store_id,cycle_idx_week1,cycle_idx_week2,cycle_idx_week4,max,min,type_of_business,1st_max,2nd_max,3rd_max,interval_M_mean,interval_M_median,interval_M_std,interval_M_75,interval_D_mean,interval_D_median,interval_D_std,tr_len
0,0,1,0,0,2019-02-28,2016-06-01,기타 미용업,159.816667,136.766667,119.850000,323.152828,86.5,646.610134,206.25,1.249377,1.0,0.647790,1002.0
1,1,2,0,0,2019-02-28,2016-06-01,NaN,259.583333,143.783333,141.466667,380.918107,50.0,941.022028,125.00,1.491071,1.0,1.035831,1002.0
2,2,36,18,1,2019-02-25,2016-06-11,NaN,594.483333,547.383333,528.333333,2300.888693,205.0,4878.540612,2631.00,3.609489,2.0,5.206088,989.0
3,4,0,0,0,2019-02-28,2016-06-01,NaN,89.533333,88.650000,71.883333,248.022708,53.0,505.312710,125.00,1.123318,1.0,0.388434,1002.0
4,5,6,0,0,2019-02-28,2016-06-01,의복 액세서리 및 모조 장신구 도매업,212.100000,211.633333,188.533333,739.295337,109.0,1288.336559,1212.50,1.522796,1.0,1.061401,1002.0


In [0]:
store_info.columns

Index(['store_id', 'cycle_idx_week1', 'cycle_idx_week2', 'cycle_idx_week4',
       'max', 'min', 'type_of_business', '1st_max', '2nd_max', '3rd_max',
       'interval_M_mean', 'interval_M_median', 'interval_M_std',
       'interval_M_75', 'interval_D_mean', 'interval_D_median',
       'interval_D_std', 'tr_len'],
      dtype='object')

In [0]:
store_info.to_csv('data/comp_9th/store_info.csv',index=False)

In [0]:

full_region	type_of_business	type_bus	high_region

train_raw = pd.read_csv("./data/comp_9th/fully_dropped.csv", parse_dates=['transacted_date','t_DateTime'], index_col=0)
train_raw['Time'] = pd.to_timedelta(train_raw['Time'])
train_raw['t_time_interval'] = pd.to_timedelta(train_raw['t_time_interval'])

naive_store_info = train_raw.groupby('store_id')['region','type_of_business'].max().reset_index()
naive_store_info['summarized_bus'] = naive_store_info['type_of_business'].apply(lambda x: x.split()[-1] if pd.notna(x) else np.nan)
naive_store_info['h_level_region'] = naive_store_info['region'].apply(lambda x: x.split()[0] if pd.notna(x) else np.nan)
naive_store_info.columns = ['store_id','full_region','type_of_business','type_bus','high_region']

naive_store_info.to_csv('data/comp_9th/naive_store_info.csv',index=False)


# 4. Derive features 4 time series by resampling


In [0]:
train_raw = pd.read_csv("./data/comp_9th/fully_dropped.csv", parse_dates=['transacted_date','t_DateTime'], index_col=0)
train_raw['Time'] = pd.to_timedelta(train_raw['Time'])
train_raw['t_time_interval'] = pd.to_timedelta(train_raw['t_time_interval'])

train_raw['point'] = train_raw['installment_term'].apply(lambda x: True if x >= 60 else False )  
train_raw['is_installment'] = train_raw['installment_term'].apply(lambda x: False if (x == 60) or ( x == 0) else True)  


trDate_store = train_raw.groupby(['store_id','transacted_date'])['amount','point','is_installment'].agg(['sum','count']).reset_index()
trDate_store.columns = ['store_id','transacted_date','sum_amt','cnt_amt','sum_pt','cnt_pt','sum_inst','cnt_inst']
trDate_store = trDate_store[['store_id','transacted_date','sum_amt','cnt_amt','sum_pt','sum_inst']]

tr_start_date = trDate_store.groupby('store_id')['transacted_date'].min().reset_index()

d = [i.date() for i in pd.date_range('2016-06-01', '2019-02-28')]

temp = (trDate_store.groupby('store_id')['sum_amt']
      .apply(lambda x: x.reindex(d)  
                        .fillna(0)
                        )   
        .reset_index()
        .rename(columns={'level_1':'transacted_date',
                          'sum_amt':'none'}))
temp['transacted_date'] = pd.to_datetime(temp['transacted_date']) 
trDate_store = temp.merge(trDate_store, on=['store_id', 'transacted_date'], how='left')
del temp


holiday = pd.read_csv("./data/comp_9th/holiday.csv", parse_dates=['h_date'])

holiday_date = holiday['h_date'].values
trDate_store['is_holiday'] = trDate_store['transacted_date'].isin(holiday_date)

holiday.columns = ['transacted_date', 'hday_name']


print(holiday[holiday.transacted_date == pd.to_datetime('2017-10-03')])

print(holiday.shape)
holiday.drop([31], inplace = True)
print(holiday.shape)

print(trDate_store.shape)
trDate_store = trDate_store.merge(holiday, how = 'left', on= 'transacted_date')
print(trDate_store.shape)

trDate_store.drop('none',axis=1,inplace = True)

trDate_store[['transacted_date','sum_amt','cnt_amt','sum_pt','sum_inst']] = trDate_store[['transacted_date','sum_amt','cnt_amt','sum_pt','sum_inst']].fillna(0)

trDate_store.columns = ['store_id', 'date', 'amount', 'tr_cnt', 'pt_cnt', 'inst_cnt', 'holiday', 'hday_name']

print(trDate_store.shape)
trDate_store = trDate_store.merge(tr_start_date,on='store_id',how='left')
print(trDate_store.shape)

# (1972901, 8)
# (1972901, 9)


print(trDate_store.shape)
trDate_store = trDate_store[trDate_store.date >= trDate_store.transacted_date]
print(trDate_store.shape)

# (1972901, 9)
# (1843654, 9)

trDate_store.drop('transacted_date',axis=1,inplace = True)
trDate_store['cur_zero'] = trDate_store['amount'].apply(lambda x: True if x ==0. else False)


trDate_store['tr_cnt'] = trDate_store['tr_cnt'].astype(int)
trDate_store['pt_cnt'] = trDate_store['pt_cnt'].astype(int)
trDate_store['inst_cnt'] = trDate_store['inst_cnt'].astype(int)



/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


   transacted_date hday_name
31      2017-10-03       개천절
32      2017-10-03        추석
(74, 2)
(73, 2)
(1972901, 8)
(1972901, 9)
(1972901, 8)
(1972901, 9)
(1972901, 9)
(1843654, 9)


In [0]:
resampled = trDate_store.copy()

In [0]:
import gc
d = [i.date() for i in pd.date_range('2016-06-01', '2019-02-28')]

rolling_days = [int(i*7) for i in range(1,15)] # 1 2 3 4 5 6 7 8 
for r_days in rolling_days:
    resampled= resampled.set_index('date')
    temp = (resampled.groupby('store_id')['amount']
                    .apply(lambda x: x.reindex(d)
                                      .fillna(0)
                                      .rolling(r_days)
                                      .sum()) # do a rolling sum
        .reset_index()
        .rename(columns={'amount': 'amount'  + '_sum_'+str(r_days)+'_days', 
                          'level_1': 'date'}))
    #temp['store_id'] = resampled.store_id.values
    temp['date'] = pd.to_datetime(temp['date']) 
    resampled = resampled.reset_index() 
    resampled = resampled.merge(temp, on=['store_id', 'date'], how = 'left')
    del temp
    gc.collect()

In [0]:
resampled['amount_0_week'] = resampled['amount_sum_7_days']

for i in range(7,92,7):
    resampled[f'amount_{int(i/7)}_week'] = resampled[f'amount_sum_{int(i+7)}_days']-resampled[f'amount_sum_{i}_days']

for i in range(13):
    resampled[f'amount_{i}{i+1}_ratio'] = (resampled[f'amount_{i}_week'] - resampled[f'amount_{i+1}_week']) / resampled[f'amount_{i+1}_week']


In [0]:
# 4 week
resampled['amount_0to4_week'] = resampled['amount_sum_28_days']
resampled['amount_4to8_week'] = resampled['amount_sum_56_days'] -  resampled['amount_sum_28_days']
resampled['amount_8to12_week'] = resampled['amount_sum_84_days'] -  resampled['amount_sum_56_days']

resampled['amount_48_ratio'] = (resampled['amount_0to4_week'] - resampled['amount_4to8_week']) /  resampled['amount_4to8_week']
resampled['amount_812_ratio'] = (resampled['amount_4to8_week'] - resampled['amount_8to12_week']) /  resampled['amount_8to12_week']

# 3 week
resampled['amount_0to3_week'] = resampled['amount_sum_21_days']
resampled['amount_3to6_week'] = resampled['amount_sum_42_days'] -  resampled['amount_sum_21_days']
resampled['amount_6to9_week'] = resampled['amount_sum_63_days'] -  resampled['amount_sum_42_days']
resampled['amount_9to12_week'] = resampled['amount_sum_84_days'] -  resampled['amount_sum_63_days']

resampled['amount_36_ratio'] = (resampled['amount_0to3_week'] - resampled['amount_3to6_week']) /  resampled['amount_3to6_week']
resampled['amount_69_ratio'] = (resampled['amount_3to6_week'] - resampled['amount_6to9_week']) /  resampled['amount_6to9_week']
resampled['amount_912_ratio'] = (resampled['amount_6to9_week'] - resampled['amount_9to12_week']) /  resampled['amount_9to12_week']

# 6 7 
resampled['amount_6to12_week']= resampled['amount_sum_84_days'] - resampled['amount_sum_42_days']
resampled['amount_7to14_week'] = resampled['amount_sum_98_days'] -  resampled['amount_sum_49_days']

resampled['amount_612_ratio'] = (resampled['amount_sum_42_days'] - resampled['amount_6to12_week']) / (resampled['amount_6to12_week'])
resampled['amount_714_ratio'] = (resampled['amount_sum_49_days'] - resampled['amount_7to14_week']) / (resampled['amount_7to14_week'])


In [0]:
print(len(resampled.columns))
resampled.drop(['amount_sum_7_days','amount_sum_21_days','amount_sum_28_days','amount_sum_98_days'], axis=1, inplace= True)
print(len(resampled.columns))

66
62


In [0]:
d = [i.date() for i in pd.date_range('2016-06-01', '2019-02-28')]

rolling_days = [7, 14 , 21, 28, 56, 84, 91] # 1 2 3 4 8 12 13 

for r_days in rolling_days:
    resampled= resampled.set_index('date')
    temp = (resampled.groupby('store_id')['tr_cnt','pt_cnt','inst_cnt','holiday','cur_zero']
                    .apply(lambda x: x.reindex(d)
                                      .fillna(0)  
                                      .rolling(r_days)
                                      .sum()) 
        .reset_index()
        .rename(columns={ 'tr_cnt':   'tr_cnt'  + '_sum_'+str(r_days)+'_days', 
                          'pt_cnt':   'pt_cnt'  + '_sum_'+str(r_days)+'_days', 
                          'inst_cnt': 'inst_cnt'  + '_sum_'+str(r_days)+'_days', 
                          'holiday':  'holiday'  + '_sum_'+str(r_days)+'_days',
                          'cur_zero': 'cur_zero' + '_sum_'+str(r_days)+'_days',
                          'level_1': 'date'}))
    temp['date'] = pd.to_datetime(temp['date'])
    resampled = resampled.reset_index() 
    resampled = resampled.merge(temp, on=['store_id', 'date'])
    del temp
    gc.collect()

In [0]:
d = [i.date() for i in pd.date_range('2016-06-01', '2019-02-28')]

rolling_days = [7, 14, 21, 28, 56, 84, 91] # 1 2 3 4 8 12 13 

for r_days in rolling_days:
    resampled= resampled.set_index('date')
    week = int(r_days/7)
    temp = (resampled.groupby('store_id')['amount_sum_91_days']
                      .apply(lambda x: x.reindex(d)  
                                        .fillna(0)  
                                        .rolling(r_days)
                                        .mean()) 
        .reset_index()
        .rename(columns={'amount_sum_91_days': 'amount_sum_91'+'_avg_'+str(week)+'_weeks',
                          'level_1': 'date'}))
    temp['date'] = pd.to_datetime(temp['date']) 
    resampled = resampled.reset_index() 
    resampled = resampled.merge(temp, on=['store_id', 'date'])
    del temp
    gc.collect()

In [0]:
resampled.shape # (1843654, 76)

(1843654, 104)

In [0]:
import re
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
        
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear','weekofyear']
    for n in attr: 
        df[targ_pre + n] = getattr(fld.dt, n.lower())
        
    if drop: 
        df.drop(fldname, axis=1, inplace=True)

d = [i.date() for i in pd.date_range('2016-06-01', '2019-05-31')]
resampled = resampled.set_index('date')
temp = (resampled.groupby('store_id')['amount']
      .apply(lambda x: x.reindex(d)
                        .fillna(0))   
      .reset_index()
      .rename(columns={'level_1':'date',
                        'amount':'amount'}))

print(temp.shape)
add_datepart(temp,'date',False)
print(temp.shape)

holiday_date = holiday['transacted_date'].values
print(temp.shape)
temp['is_holiday'] = temp['date'].isin(holiday_date)
print(temp.shape)

(2153865, 3)
(2153865, 10)
(2153865, 10)
(2153865, 11)


In [0]:
temp['is_weekend'] = temp['Dayofweek'].apply(lambda x: True if (x == 5) or (x== 6) else False)
temp = temp.set_index('date')

In [0]:
# 0 월요일 1 2 3 4 5 6
d = [i.date() for i in pd.date_range('2016-06-01', '2019-05-31')]
d = d[::-1]
r_days = 92
temp_date_info = (temp.iloc[::-1].groupby('store_id')['is_weekend','is_holiday','amount']
                  .apply(lambda x: x.reindex(d)  
                                    .fillna(0)  
                                    .rolling(r_days)
                                    .sum()
                                    .iloc[::-1]) 
          .reset_index()
          .rename(columns={'amount': 'y',
                          'is_weekend': 'weekend_sum',
                          'is_holiday': 'h_day_sum',
                          'level_1': 'date'}))


In [0]:
temp = temp.reset_index()
temp_date_info['date'] = pd.to_datetime(temp_date_info['date'])
print(temp.shape)
temp = temp.merge(temp_date_info,on=['date', 'store_id'], how = 'left')
print(temp.shape)

# (2153865, 12)
# (2153865, 15)


(2153865, 12)
(2153865, 15)


In [0]:
temp['y'] = temp['y'] - temp['amount']
temp['weekend_sum'] = temp['weekend_sum'] - temp['is_weekend']
temp['h_day_sum'] = temp['h_day_sum'] - temp['is_holiday']

In [0]:
resampled = resampled.reset_index()
temp.drop('amount',axis=1,inplace=True)

In [0]:
print(resampled.shape)
resampled = resampled.merge(temp, on=['date', 'store_id'], how = 'left')
print(resampled.shape)

# (1843654, 104)
# (1843654, 116)

(1843654, 104)
(1843654, 116)


In [0]:
resampled.head()

,date,store_id,amount,tr_cnt,pt_cnt,inst_cnt,holiday,hday_name,cur_zero,amount_sum_14_days,amount_sum_35_days,amount_sum_42_days,amount_sum_49_days,amount_sum_56_days,amount_sum_63_days,amount_sum_70_days,amount_sum_77_days,amount_sum_84_days,amount_sum_91_days,amount_0_week,amount_1_week,amount_2_week,amount_3_week,amount_4_week,amount_5_week,amount_6_week,amount_7_week,amount_8_week,amount_9_week,amount_10_week,amount_11_week,amount_12_week,amount_13_week,amount_01_ratio,amount_12_ratio,amount_23_ratio,amount_34_ratio,amount_45_ratio,amount_56_ratio,amount_67_ratio,...,cur_zero_sum_21_days,tr_cnt_sum_28_days,pt_cnt_sum_28_days,inst_cnt_sum_28_days,holiday_sum_28_days,cur_zero_sum_28_days,tr_cnt_sum_56_days,pt_cnt_sum_56_days,inst_cnt_sum_56_days,holiday_sum_56_days,cur_zero_sum_56_days,tr_cnt_sum_84_days,pt_cnt_sum_84_days,inst_cnt_sum_84_days,holiday_sum_84_days,cur_zero_sum_84_days,tr_cnt_sum_91_days,pt_cnt_sum_91_days,inst_cnt_sum_91_days,holiday_sum_91_days,cur_zero_sum_91_days,amount_sum_91_avg_1_weeks,amount_sum_91_avg_2_weeks,amount_sum_91_avg_3_weeks,amount_sum_91_avg_4_weeks,amount_sum_91_avg_8_weeks,amount_sum_91_avg_12_weeks,amount_sum_91_avg_13_weeks,Year,Month,Week,Day,Dayofweek,Dayofyear,weekofyear,is_holiday,is_weekend,weekend_sum,h_day_sum,y
0,2016-06-01,0,12571.428571,4,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,1,2,153,22,False,False,26.0,3.0,2.611000e+06
1,2016-06-02,0,40571.428571,7,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,2,3,154,22,False,False,26.0,3.0,2.588000e+06
2,2016-06-03,0,18142.857143,3,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,3,4,155,22,False,False,26.0,3.0,2.594571e+06
3,2016-06-04,0,31714.285714,7,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,4,5,156,22,False,True,26.0,3.0,2.624143e+06
4,2016-06-05,0,10428.571429,3,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,5,6,157,22,False,True,26.0,3.0,2.662143e+06


In [0]:
resampled.iloc[-91:].amount.sum()
# 6616071.428571428

6616071.428571428

# 5. cycle idx

In [0]:
print(resampled.shape)

(1843654, 116)


In [0]:
resampled_pos = resampled[resampled.amount > 0.].copy()
print(resampled_pos.shape)

(1283048, 116)


In [0]:
resampled_pos = resampled_pos[['date','store_id','amount','y']]

In [0]:
resampled_pos['nxt_tr_week']  = resampled_pos.groupby(['store_id'])['date'].shift(-1)
resampled_pos['tr_interval'] = (resampled_pos['nxt_tr_week'] - resampled_pos['date'])/ np.timedelta64(1, 'D')

resampled_pos['tr_interval'] = resampled_pos['tr_interval'].fillna(value = 0.)

In [0]:
resampled_pos['b4_tr_week']  = resampled_pos.groupby(['store_id'])['date'].shift(1)
resampled_pos['b4_tr_interval'] = (resampled_pos['date'] - resampled_pos['b4_tr_week'])/ np.timedelta64(1, 'D')

resampled_pos['b4_tr_interval'] = resampled_pos['b4_tr_interval'].fillna(value = 0.)

In [0]:
MAX_INTERVAL_DATE = [7,28,56] # 7 28 56

def make_cycle(df):
    for i in MAX_INTERVAL_DATE:
        df['is_closed'] = df['tr_interval'].map(lambda x: True if ((x == 0.) or (x > i)) else False  )

        df[f'cycle_idx_week{int((i)/7) }'] = (df.groupby('store_id')['is_closed']
                                                  .cumsum()
                                                  .shift(1)
                                                  .fillna(0)
                                                  .astype(int)
                                                    )

        head_index = df.groupby('store_id')[f'cycle_idx_week{int((i+1)/7)}'].head(1).index
        df.loc[head_index, f'cycle_idx_week{int((i+1)/7)}'] = 0 
    return df

resampled_pos = make_cycle(resampled_pos)


In [0]:
len(resampled_pos[resampled_pos.b4_tr_interval == 0.])

1967

In [0]:
len(resampled_pos[resampled_pos.b4_tr_interval < 0.])

0

In [0]:
print(resampled_pos.shape)
resampled_pos = resampled_pos[resampled_pos.b4_tr_interval > 0.]
print(resampled_pos.shape)

(1283048, 12)
(1281081, 12)


In [0]:
resampled_pos = resampled_pos[['date','store_id','b4_tr_interval','cycle_idx_week1','cycle_idx_week4','cycle_idx_week8']]



In [0]:
resampled = resampled.merge(resampled_pos,how = 'left' , on = ['date','store_id'])

In [0]:
print(resampled.shape)

(1843654, 120)


In [0]:
def mask_first(x):
    result = np.ones_like(x)
    result[0] = 0
    return result

masked = resampled.groupby(['store_id'])['store_id'].transform(mask_first).astype(bool)
resampled = resampled.loc[masked]

In [0]:
print(resampled.shape)

(1841687, 120)


In [0]:
new_feat = ['b4_tr_interval','cycle_idx_week1','cycle_idx_week4','cycle_idx_week8']
resampled[new_feat] = resampled[new_feat].fillna(-1)

In [0]:
!dir data

comp_9th	    resampled_existing_0821.csv  trDate_store_0824.csv
resampled_0824.csv  resampled_w0_0821.csv


In [0]:
resampled.to_csv('./data/resampled_0825.csv',index=False)

In [0]:
resampled.head()

,date,store_id,amount,tr_cnt,pt_cnt,inst_cnt,holiday,hday_name,cur_zero,amount_sum_14_days,amount_sum_35_days,amount_sum_42_days,amount_sum_49_days,amount_sum_56_days,amount_sum_63_days,amount_sum_70_days,amount_sum_77_days,amount_sum_84_days,amount_sum_91_days,amount_0_week,amount_1_week,amount_2_week,amount_3_week,amount_4_week,amount_5_week,amount_6_week,amount_7_week,amount_8_week,amount_9_week,amount_10_week,amount_11_week,amount_12_week,amount_13_week,amount_01_ratio,amount_12_ratio,amount_23_ratio,amount_34_ratio,amount_45_ratio,amount_56_ratio,amount_67_ratio,...,holiday_sum_28_days,cur_zero_sum_28_days,tr_cnt_sum_56_days,pt_cnt_sum_56_days,inst_cnt_sum_56_days,holiday_sum_56_days,cur_zero_sum_56_days,tr_cnt_sum_84_days,pt_cnt_sum_84_days,inst_cnt_sum_84_days,holiday_sum_84_days,cur_zero_sum_84_days,tr_cnt_sum_91_days,pt_cnt_sum_91_days,inst_cnt_sum_91_days,holiday_sum_91_days,cur_zero_sum_91_days,amount_sum_91_avg_1_weeks,amount_sum_91_avg_2_weeks,amount_sum_91_avg_3_weeks,amount_sum_91_avg_4_weeks,amount_sum_91_avg_8_weeks,amount_sum_91_avg_12_weeks,amount_sum_91_avg_13_weeks,Year,Month,Week,Day,Dayofweek,Dayofyear,weekofyear,is_holiday,is_weekend,weekend_sum,h_day_sum,y,b4_tr_interval,cycle_idx_week1,cycle_idx_week4,cycle_idx_week8
1,2016-06-02,0,40571.428571,7,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,2,3,154,22,False,False,26.0,3.0,2.588000e+06,1.0,0.0,0.0,0.0
2,2016-06-03,0,18142.857143,3,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,3,4,155,22,False,False,26.0,3.0,2.594571e+06,1.0,0.0,0.0,0.0
3,2016-06-04,0,31714.285714,7,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,4,5,156,22,False,True,26.0,3.0,2.624143e+06,1.0,0.0,0.0,0.0
4,2016-06-05,0,10428.571429,3,0,0,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,22,5,6,157,22,False,True,26.0,3.0,2.662143e+06,1.0,0.0,0.0,0.0
5,2016-06-06,0,17285.714286,4,0,0,True,현충일,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,23,6,0,158,23,True,False,26.0,2.0,2.674429e+06,1.0,0.0,0.0,0.0


- References:
    - Reducing negative value
        - https://dacon.io/index.php?mid=code1&category=58792&document_srl=58793

    - Korean Holiday Data
        - https://superelement.tistory.com/15

    - Derive features 4 time series by resampling
        - https://stackoverflow.com/questions/28642511/how-to-apply-rolling-functions-in-a-group-by-object-in-pandas
        - https://www.kaggle.com/adityaecdrid/my-first-time-series-comp-added-prophet
        - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.Rolling.sum.html

    
